In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import LoraConfig, AutoPeftModelForCausalLM, PeftModel
from trl import SFTConfig, SFTTrainer
from huggingface_hub import HfApi

- 데이터셋 분할(8:2)

In [ ]:
dataset = load_dataset("eddyfox8812/otc-mrc-ko-rag-dataset", split="train")

system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 문서에는 반드시 약물의 이름이 기재되어 있습니다. 사용자가 질문하는 약물만을 정확히 찾아 해당 문서만을 인용하여 답변하십시오.
6. 반드시 사용자가 질문한 약물에 관한 문서로만 답변해야 합니다. 약물이 다른 문서를 절대로 인용하지 마십시오

검색 결과:
-----
{search_result}"""

print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

test_ratio = 0.2

train_data = []
test_data = []

for type_name in set(dataset['type']):
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]
     
    test_size = int(len(curr_type_data) * test_ratio)
    
    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

def format_data(sample):
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])
    
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

README.md:   0%|          | 0.00/433 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/990 [00:00<?, ? examples/s]

원본 데이터의 type 분포:
no_answer: 328
mrc_question: 662

전체 데이터 분할 결과: Train 793개, Test 197개

학습 데이터의 type 분포:
no_answer: 263
mrc_question: 530

테스트 데이터의 type 분포:
no_answer: 65
mrc_question: 132


In [ ]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 문서에는 반드시 약물의 이름이 기재되어 있습니다. 사용자가 질문하는 약물만을 정확히 찾아 해당 문서만을 인용하여 답변하십시오.\n6. 반드시 사용자가 질문한 약물에 관한 문서로만 답변해야 합니다. 약물이 다른 문서를 절대로 인용하지 마십시오\n\n검색 결과:\n-----\n문서1: 옵티노즈연질캡슐 :\n\n### 주의사항(1)\n5. 다음과 같은 경우 이 약의 복용을 즉각 중지하고 의사, 치과의사, 약사와 상의할 것. 상담 시 가능한 한 이 첨\n부문서를 소지할 것.\n1) 이 약의 복용에 의해 다음의 증상이 나타난 경우\n발진·발적(충혈되어 붉어짐), 가려움, 구역·구토, 식욕부진, 변비, 부종(부기), 배뇨(소변을 눔)곤란, 목마름\n(지속적이거나 심한), 어지러움, 불안, 떨림, 불면\n2) 이 약의 복용에 의해 드물게 아래의 중증(심한 증상) 증상이 나타난 경우\n① 쇽(아나필락시)(과민성쇼크) : 복용후 바로 두드러기, 부종(부기), 가슴답답함 등과 함께 안색이 창백하\n\n① 쇽(아나필락시)(과민성쇼크) : 복용후 바로 두드러기, 부종(부기), 가슴답답함 등과 함께 안색이 창백하\n고, 손발이 차고, 식은땀, 숨쉬기 곤란함 등이 나타날 수 있다.\n② 피부점막안증후군(스티븐스-존슨증후군), 중독성표피괴사용해(리엘증후군) : 고열을 동반하고, 발진·발

In [ ]:
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/197 [00:00<?, ? examples/s]

In [ ]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct" 

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

In [ ]:
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 문서에는 반드시 약물의 이름이 기재되어 있습니다. 사용자가 질문하는 약물만을 정확히 찾아 해당 문서만을 인용하여 답변하십시오.
6. 반드시 사용자가 질문한 약물에 관한 문서로만 답변해야 합니다. 약물이 다른 문서를 절대로 인용하지 마십시오

검색 결과:
-----
문서1: 복합마이녹실액5% :

### 주의사항(1)
6. 저장상의 주의사항
1) 이 약은 인화성이 있으므로 화기를 피한다.
2) 어린이의 손이 닿지 않는 곳에 보관한다.
3) 의약품을 원래 용기에서 꺼내어 다른 용기에 보관하는 것은 의약품 오용에 따른 사고 발생이나 의약품 품
질저하의 원인이 될 수 있으므로 원래의 용기에 넣고 꼭 닫아 보관한다.
-----
문서2: 메디녹실액5%(미녹시딜) :

### 주의사항(1)
6. 저장상의 주의사항
1) 이 약은 인화성이 있으므로 화기를 피한다.
2) 어린이의 손이 닿지 않는 곳에 보관한다.
3) 의약품을 원래 용기에서 꺼내어 다른 용기에 보관하는 것은 의약품 오용에 따른 사고 발생이나 의약품 품
질저하의 원인이 될 수 있으므로 원래의 용기에 넣고 꼭 닫아 보관한다.
-----
문서3: 카필러스액5%(미녹시딜) :

### 주의사항(1)
6. 저장상의 주의사항
1) 이 약은 인화성이 있으므로 화기를 피한다.
2) 어린이의 

In [ ]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [ ]:
args = SFTConfig(
    output_dir="llama-3-8b-otc-rag-ko",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    bf16=True,
    learning_rate=1e-4, 
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

In [ ]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        text = prompt.strip()

        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j] 
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

In [ ]:
max_seq_length=8192

example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 981])
어텐션 마스크 형태: torch.Size([1, 981])
레이블 형태: torch.Size([1, 981])


In [ ]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 115036, 99901, 18918, 82818, 120378, 43139, 109760, 19954, 111964, 110513, 109670, 382, 13447, 49531, 21028, 67890, 30426, 115790, 18359, 103386, 100968, 119978, 627, 16, 13, 109760, 54780, 115036, 99901, 18918, 82818, 120378, 43139, 111964, 16582, 119978, 627, 17, 13, 115036, 99901, 19954, 108838, 109842, 18359, 111964, 16582, 113348, 117193, 96677, 119978, 627, 18, 13, 109760, 19954, 102597, 108386, 13094, 115036, 99901, 19954, 47782, 115300, 115036, 99901, 102772, 330, 34983, 65895, 109760, 93, 19954, 102597, 109842, 13094, 120078, 1210, 103959, 35495, 111964, 16582, 119978, 627, 19, 13, 111964, 48936, 54718, 103966, 30381, 117294, 18918, 119884, 83290, 54535, 41953, 108520, 54535, 101353, 18359, 114839, 101528, 33390, 107333, 19954, 102722, 102657, 16969, 23955, 101711, 84734, 17835, 95713, 117294, 85721, 48424, 18918, 102484, 21121, 119978, 13, 96717, 18918, 105510, 27796, 103966, 30381, 54535, 41953, 10659

In [ ]:
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 문서에는 반드시 약물의 이름이 기재되어 있습니다. 사용자가 질문하는 약물만을 정확히 찾아 해당 문서만을 인용하여 답변하십시오.
6. 반드시 사용자가 질문한 약물에 관한 문서로만 답변해야 합니다. 약물이 다른 문서를 절대로 인용하지 마십시오

검색 결과:
-----
문서1: 복합마이녹실액5% :

### 주의사항(1)
6. 저장상의 주의사항
1) 이 약은 인화성이 있으므로 화기를 피한다.
2) 어린이의 손이 닿지 않는 곳에 보관한다.
3) 의약품을 원래 용기에서 꺼내어 다른 용기에 보관하는 것은 의약품 오용에 따른 사고 발생이나 의약품 품
질저하의 원인이 될 수 있으므로 원래의 용기에 넣고 꼭 닫아 보관한다.
-----
문서2: 메디녹실액5%(미녹시딜) :

### 주의사항(1)
6. 저장상의 주의사항
1) 이 약은 인화성이 있으므로 화기를 피한다.
2) 어린이의 손이 닿지 않는 곳에 보관한다.
3) 의약품을 원래 용기에서 꺼내어 다른 용기에 보관하는 것은 의약품 오용에 따른 사고 발생이나 의약품 품
질저하의 원인이 될 수 있으므로 원래의 용기에 넣고 꼭 닫아 보관한다.
-----
문서3: 카필러스액5%(미녹시딜) :

### 주의사항(1)
6. 저장상의 주의사항
1) 이 약은 인화성이 있으므로

In [ ]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [ ]:
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
검색 결과에는 복합마이녹실액5%와 다른 약물의 상호작용을 찾을 수 없습니다.<|eot_id|>


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [ ]:
trainer.train()
trainer.save_model()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.021500
20,0.425000
30,0.327800
40,0.296800
50,0.242200
60,0.250100
70,0.213600
80,0.222700
90,0.238700
100,0.217000


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-pac

In [ ]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [ ]:
print(prompt_lst[100])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 문서에는 반드시 약물의 이름이 기재되어 있습니다. 사용자가 질문하는 약물만을 정확히 찾아 해당 문서만을 인용하여 답변하십시오.
6. 반드시 사용자가 질문한 약물에 관한 문서로만 답변해야 합니다. 약물이 다른 문서를 절대로 인용하지 마십시오

검색 결과:
-----
문서1: 욱씬정 :

### 주의사항(1)
1. 경고
1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열진통제를 복용해야 할
경우 반드시 의사 또는 약사와 상의해야 한다. 이러한 사람이 이 약을 복용하면 간손상이 유
발될 수 있다.
2) 아세트아미노펜을 복용한 환자에서 매우 드물게 급성 전신성 발진성 농포증(급성 전신성 발
진성 고름물집증)(AGEP), 스티븐스 - 존슨 증후군(SJS), 독성 표피 괴사용해(TEN)와 같은 중대
한 피부 반응이 보고되었고, 이러한 중대한 피부반응은 치명적일 수 있다. 따라서 이러한 중대한
피부반응의 징후에 대하여 환자들에게 충분히 알리고, 이 약 투여 후 피부발진이나 다른 과민반
응의 징후가 나타나면 즉시 복용을 중단하도록 하여야 한다.
3) 이 약은 아세트아미노펜을 함유하고 있다. 아세트아미노펜으로 일일 최대 용량(4,000mg)을
초과할 경우 간손상을 일으킬 수 있으므로 이 약을 일일 최대 용량(4000mg

In [ ]:
print(label_lst[100])


자니틴정(니자티딘)을 복용하는 동안에는 알코올 음료를 마시는 것이 금지됩니다. [[ref5]]


- base model에 붙여 확인

In [ ]:
base_model_name = "NCSOFT/Llama-VARCO-8B-Instruct"  
peft_model_id = "llama-3-8b-otc-rag-ko/checkpoint-594"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map={"": 0},  # GPU 0번
)
fine_tuned_model = PeftModel.from_pretrained(
    base_model,
    peft_model_id,
)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [ ]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

In [ ]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
for prompt, label in zip(prompt_lst[100:103], label_lst[100:103]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


    response:
자니틴정 복용 중에는 알코올 음주가 금지됩니다. 이 약을 복용하는 동안에는 알코올 음료를 마시는 것을 피해야 합니다[[ref5]].
    label:

자니틴정(니자티딘)을 복용하는 동안에는 알코올 음료를 마시는 것이 금지됩니다. [[ref5]]
--------------------------------------------------
    response:
타라부틴정 복용 시 드물게 나타날 수 있는 소화기계 이상반응은 변비, 설사, 복명(창자 가스소리), 구역, 구토, 소화장애, 구갈(목마름), 구내마비감 등입니다[[ref1]].
    label:

타라부틴정 복용 시 드물게 나타날 수 있는 소화기계 이상반응으로는 변비, 설사, 복명(창자 가스소리), 구역, 구토, 소화장애, 구갈(목마름), 구내(입안)마비감 등이 있습니다[[ref1]].
--------------------------------------------------
    response:
써큐록신정 120밀리그램의 성인 용법은 말초동맥 순환장애, 어지러움, 이명 등의 증상을 치료하기 위해 1회 40mg을 1일 3회 또는 1회 80mg을 1일 2회 경구투여하는 것입니다. 또한, 기질성 뇌기능 장애를 다루기 위해서는 1회 40 ~ 80mg을 1일 3회 또는 1회 120mg을 1일 2회 경구 투여합니다. 연령과 증상에 따라 적절히 증감할 수 있습니다[[ref3]].
    label:

써큐록신정 120밀리그램의 성인 용법은 다음과 같습니다. 말초동맥 순환장애, 어지러움, 이명에 대해서는 1회 40mg을 1일 3회 또는 1회 80mg을 1일 2회 경구 투여하며, 기질성 뇌기능 장애에 대해서는 1회 40 ~ 80mg을 1일 3회 또는 1회 120mg을 1일 2회 경구 투여합니다. 연령과 증상에 따라 적절히 증감할 수 있습니다[[ref3]].
--------------------------------------------------


- 모델, 토크나이저 저장 후 test

In [ ]:
base_model_path = "NCSOFT/Llama-VARCO-8B-Instruct"
adapter_path = "./llama-3-8b-otc-rag-ko/checkpoint-594"
merged_model_path = "./output_dir"

device_arg = {"device_map": "auto"}

print(f"Loading base model from: {base_model_path}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)

print(f"Loading and merging PEFT from: {adapter_path}")
model = PeftModel.from_pretrained(base_model, adapter_path, **device_arg)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(base_model_path)

print(f"Saving merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("✅ 모델과 토크나이저 저장 완료")

Loading base model from: NCSOFT/Llama-VARCO-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading and merging PEFT from: ./llama-3-8b-otc-rag-ko/checkpoint-594
Saving merged model to: ./output_dir
✅ 모델과 토크나이저 저장 완료


In [ ]:
model = 'NCSOFT/Llama-VARCO-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model)

prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [ ]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
model_id = './output_dir'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

for prompt, label in zip(prompt_lst[150:155], label_lst[150:155]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

    response:
훼리맘큐연질캡슐은 12세 이상의 어린이와 성인이 1일 1회, 1회 1캡슐을 복용해야 합니다[[ref2]].
    label:

훼리맘큐연질캡슐은 12세 이상 어린이 및 성인이 1일 1회, 1회 1캡슐을 복용하도록 되어 있습니다[[ref2]].
--------------------------------------------------
    response:
포머렐정은 어린이의 손이 닿지 않는 곳에 보관해야 하며, 다른 용기에 바꾸어 넣는 것은 사고의 원인이 될 수 있으므로 주의해야 합니다. [[ref1]]
    label:

포머렐정은 어린이의 손이 닿지 않는 곳에 보관해야 하며, 의약품을 원래 용기에서 꺼내어 다른 용기에 보관하는 것은 사고의 원인이 되거나 품질 유지에 바람직하지 않으므로 주의해야 합니다[[ref1]].
--------------------------------------------------
    response:
니자티드정은 성인이 하루에 75 mg을 경구투여할 수 있으며, 증상의 개선이 없을 경우 추가로 75 mg을 투여할 수 있습니다. 그러나 1일 최대 복용량은 150 mg을 초과할 수 없습니다[[ref5]].
    label:

니자티드정(니자티딘)의 경우, 성인은 하루에 1회 75 mg을 복용할 수 있으며, 증상의 개선이 없을 경우 8시간 후에 추가로 75 mg을 투여할 수 있습니다. 그러나 1일 최대 복용량은 150 mg을 초과할 수 없습니다[[ref5]].
--------------------------------------------------
    response:
가스키환은 임신 가능성이 있는 여성에게 복용하지 말아야 합니다. 이 약은 신생아, 수유부, 임부 또는 임신하고 있을 가능성이 있는 여성에게 복용하지 말아야 한다고 명시되어 있습니다[[ref5]].
    label:

가스키환은 임부 또는 임신하고 있을 가능성이 있는 여성에게 복용하지 말아야 합니다. 이 약물은 이러한 여성에게 안

- 업로드

In [ ]:
api = HfApi()

username = "eddyfox8812"

In [ ]:
MODEL_NAME = "llama-3-8b-otc-rag-ko-checkpotint-594"

In [ ]:
api.create_repo(
    token="api key",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token="api key",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir",
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            